# 使用 LoRA 对模型进行微调

## LoRA 原理

**LoRA（Low-Rank Adaptation，低秩自适应）** 是一种高效微调大语言模型（LLM）的技术，其核心思想是通过低秩分解，在原始模型参数旁添加少量可训练的“低秩矩阵”来模拟参数更新，而非直接调整庞大的原始参数。这种方法大幅减少了训练参数量（通常降低万倍以上），显著节省计算资源和存储开销，同时保持与全参数微调相近的效果，且推理时可通过合并矩阵避免额外延迟。LoRA特别适合资源有限或需要快速适配多任务的大模型场景。其原理图如下所示：

![LoRA Struct](img/LoRA-Struct.svg)

（图中左侧表示全参数微调，右侧表示 LoRA 微调，蓝色部分表示需要训练的权重，灰色部分表示冻结的权重不参与训练）

经过 LoRA 微调后的模型结果会产生一个额外的 LoRA 权重对应于图中的虚线框部分，下次调用模型时将 LoRA 权重加载到基础模型上即可。

## LoRA 训练

在开源模型技术蓬勃发展的当下，以 LoRA 为代表的参数高效微调方法已成为大模型适配下游任务的重要范式。尽管当前主流大语言模型的架构参数大多已开源，**但工程实践中开发者更关注微调策略的实施而非底层网络结构的实现细节**。为此，本文聚焦大语言模型的微调训练实践，基于Hugging Face生态中的强化学习训练库TRL（Transformer Reinforcement Learning），系统解析监督式微调（SFT）等核心技术的实现路径。

TRL作为🤗 Transformers生态的重要组件，集成了监督式微调（SFT）、近端策略优化（PPO）和直接偏好优化（DPO）等前沿训练范式。该库不仅支持多模态模型的灵活适配，更能通过分布式训练框架实现从单卡到集群的弹性扩展，为不同硬件环境下的模型调优提供统一的技术解决方案。其模块化设计兼顾算法创新与工程落地，开发者既可直接调用预置训练流程，也可基于API灵活构建定制化训练策略。

- TRL 开源：[Github - trl](https://github.com/huggingface/trl)
- TRL 文档：[huggingface - trl](https://hf.co/docs/trl)

### 1. 引入必要的库

In [ ]:
import torch
from transformers import (
    AutoModelForCausalLM,
    BitsAndBytesConfig,
    TrainingArguments
)
from peft import (
    LoraConfig,
    TaskType,
    get_peft_model
)
from trl import SFTConfig,SFTTrainer
from datasets import load_dataset, Dataset

### 2. 准备数据

本地已经下载好了用于训练的数据存放在 `.\dataset\Capybara` 目录下。

In [ ]:
# 可以直接从 huggingface 的库中进行下载
# dataset = load_dataset("trl-lib/Capybara", split="train")

dataset = Dataset.load_from_disk('./dataset/Capybara')

def formatting_prompts_func(example):
    output_texts = []
    for i in range(len(example['messages'])):
        text = f"### Question: {example['messages'][i]}\n ### Answer: {example['output'][i]}"
        output_texts.append(text)
    return output_texts

### 3. 使用 LoRA 的方式加载模型

下面是 `LoraConfig` 的配置参数：

| 分类                  | 参数名称                    | 类型/选项                                                                 | 说明                                                                                                                                                     | 备注                                                                                     |
|-----------------------|----------------------------|--------------------------------------------------------------------------|---------------------------------------------------------------------------------------------------------------------------------------------------------|-----------------------------------------------------------------------------------------|
| **基础配置**          | `r`                        | `int`                                                                   | LoRA 的秩（Rank），决定低秩矩阵的维度                                                                                                                     | 常用范围 8-64                                                                           |
|                       | `lora_alpha`               | `int`                                                                   | LoRA 的缩放系数，与 `r` 共同控制低秩矩阵的缩放幅度                                                                                                         | 默认缩放比例：`lora_alpha/r`（或 `lora_alpha/sqrt(r)` 若启用 `use_rslora`）               |
|                       | `lora_dropout`             | `float`                                                                 | LoRA 层的 Dropout 概率，防止过拟合                                                                                                                       | 范围 0.0-1.0，常用 0.1-0.3                                                              |
|                       | **`task_type`**            | `"CAUSAL_LM"` / `"SEQ_CLS"` / `"SEQ_2_SEQ_LM"` / `"TOKEN_CLS"` 等        | **指定模型的任务类型**，决定适配器的插入位置和训练行为                                                                                                     | 必填项，需与模型架构匹配（如 `"CAUSAL_LM"` 对应 GPT、Llama 等自回归模型）                 |
| **目标模块选择**      | `target_modules`           | `List[str]` / `str` / `"all-linear"`                                    | 指定应用 LoRA 的模块名称（支持正则匹配、后缀匹配或自动选择所有线性层）                                                                                     | 示例：`["query", "value"]`                                                               |
|                       | `exclude_modules`          | `List[str]` / `str`                                                     | 排除不需要应用 LoRA 的模块                                                                                                                               | 优先级高于 `target_modules`                                                              |
|                       | `layers_to_transform`      | `List[int]` / `int`                                                     | 指定要转换的层索引                                                                                                                                       | 示例：`[0, 1]` 表示仅修改前两层                                                           |
|                       | `layers_pattern`           | `List[str]` / `str`                                                     | 模型层结构的名称模式（如 `"h"` 对应 GPT-2 的层列表）                                                                                                      |                                                                                          |
| **高级初始化与优化**  | `init_lora_weights`        | `bool` / `"gaussian"` / `"eva"` / `"pissa"` / `"corda"` / `"loftq"` 等   | 控制 LoRA 权重初始化方式：<br>- `True`：默认初始化（B 矩阵为 0）<br>- `"eva"`：基于数据 SVD 的优化初始化<br>- `"pissa"`：快速 SVD 初始化加速收敛            | 需配合 `eva_config`/`loftq_config` 等使用                                                |
|                       | `use_rslora`               | `bool`                                                                  | 启用 Rank-Stabilized LoRA，调整缩放公式为 `lora_alpha/sqrt(r)`                                                                                           |                                                                                          |
|                       | `use_dora`                 | `bool`                                                                  | 启用 DoRA，分解权重更新为幅度和方向（提升低秩性能，但增加计算开销）                                                                                       |                                                                                          |
| **特定场景配置**      | `bias`                     | `"none"` / `"lora_only"` / `"all"`                                      | 控制是否更新偏置项                                                                                                                                       | `"all"` 或 `"lora_only"` 时需注意禁用适配器后输出可能与原模型不同                          |
|                       | `fan_in_fan_out`           | `bool`                                                                  | 若模型权重形状为 `(fan_in, fan_out)`（如 GPT-2 的 `Conv1D`），需设为 `True`                                                                              |                                                                                          |
|                       | `megatron_config`          | `dict`                                                                  | 用于 Megatron 框架的并行线性层配置                                                                                                                       | 需配合 `megatron_core="megatron.core"` 使用                                               |
|                       | `trainable_token_indices`  | `List[int]` / `dict`                                                    | 仅微调嵌入层的特定 token（如特殊标记）                                                                                                                   | 示例：`{'embed_tokens': [0, 1]}`                                                         |
| **模式扩展与自定义**  | `rank_pattern`             | `dict`                                                                  | 为不同层指定独立的秩（`r`）                                                                                                                              | 示例：`{"attention.*": 16}` 对注意力层使用秩 16                                           |
|                       | `alpha_pattern`            | `dict`                                                                  | 为不同层指定独立的缩放系数（`alpha`）                                                                                                                    |                                                                                          |
|                       | `layer_replication`        | `List[Tuple[int, int]]`                                                 | 复制原始层构建新层（扩展模型深度），每个新层独立适配器                                                                                                   | 示例：`[(0, 3)]` 表示复制第 0-3 层                                                        |
| **其他**              | `modules_to_save`          | `List[str]`                                                             | 额外训练并保存的模块（如分类头）                                                                                                                         | 示例：`["classifier"]`                                                                   |
|                       | `lora_bias`                | `bool`                                                                  | 是否启用 LoRA B 矩阵的偏置（默认关闭）                                                                                                                   |                                                                                          |

In [ ]:
# LoRA 配置
lora_config = LoraConfig(
    r=16,
    lora_alpha=32,
    lora_dropout=0.05,
    bias="none",
    target_modules=["q_proj", "v_proj"], # LoRA 注入的模块
    task_type=TaskType.CAUSAL_LM,               # 大语言模型训练使用 CAUSAL_LM
)

# 量化配置（减小模型加载和训练时的显存，但精度会下降）
quantization_config = BitsAndBytesConfig(
    load_in_8bit=True,                    # 将模型量化为 8bit 进行加载
    # load_in_4bit=True,                  # 将模型量化为 4bit 进行加载
    # bnb_4bit_quant_type="nf4",
    # bnb_4bit_compute_dtype=torch.float16,
    # bnb_4bit_use_double_quant=True
)

# 加载模型，可以为模型路径或者模型ID
# model_id = "Qwen/Qwen2.5-0.5B"
model_id = "E:/Model/Qwen/Qwen2.5-0.5B"

model = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=quantization_config,
    device_map="auto",
    trust_remote_code=True  # 确保Qwen模型兼容
)

model = get_peft_model(model, lora_config)

# 打印模型结构和预期可以训练的参数
print(model)
model.print_trainable_parameters()

### 4. 进行训练

微调配置参数 `SFTConfig` 继承自 `TrainingArguments` ，下面是其相关配置：

| 分类                     | 参数名称                | 类型/默认值                                                                 | 说明                                                                                           |
|--------------------------|-------------------------|----------------------------------------------------------------------------|-----------------------------------------------------------------------------------------------|
| **模型控制参数**         | `model_init_kwargs`     | `dict[str, Any]` / `None`                                                 | 用于 `AutoModelForCausalLM.from_pretrained` 的关键字参数（当模型以字符串形式传入时）             |
|                          | `use_liger`             | `bool` / `False`                                                          | 是否使用 Liger 内核优化模型（提升吞吐量并减少内存占用）                                          |
| **数据预处理参数**       | `dataset_text_field`    | `str` / `"text"`                                                          | 数据集中包含文本数据的列名                                                                       |
|                          | `dataset_kwargs`        | `dict[str, Any]` / `None`                                                 | 数据集预处理的可选参数（仅支持 `skip_prepare_dataset` 键）                                       |
|                          | `dataset_num_proc`      | `int` / `None`                                                            | 数据集预处理使用的进程数                                                                         |
|                          | `max_seq_length`        | `int` / `1024`                                                            | 序列最大长度（超长部分截断右端），设为 `None` 则不截断                                           |
|                          | `packing`               | `bool` / `False`                                                          | 是否将多序列打包为固定长度格式（使用 `max_seq_length` 定义长度）                                 |
|                          | `eval_packing`          | `bool` / `None`                                                           | 评估集是否打包（`None` 时与 `packing` 值相同）                                                   |
| **训练控制参数**         | `learning_rate`         | `float` / `2e-5`                                                          | AdamW 优化器的初始学习率（覆盖 `TrainingArguments` 默认值）                                      |

In [ ]:
training_args = SFTConfig(
    output_dir="./output/lora",

    num_train_epochs=3,              # 训练轮数
    learning_rate=2e-4,              # 学习率

    per_device_train_batch_size=2,   # 每个设备训练的批大小
    gradient_accumulation_steps=8,   # 累积梯度，等效 batch_size=2*8=16
    gradient_checkpointing=True,     # 启用梯度检查点（牺牲20%速度换显存）

    fp16=True,                       # 使用混合精度
    # torch_compile=False,           # 禁用模型编译（减少初始显存峰值）

    logging_steps=50,
    save_steps=500,
    eval_steps=500,
    optim="adamw_torch",             # 使用标准优化器
    warmup_ratio=0.1,                # 前 10% 步数做学习率预热
)

trainer = SFTTrainer(
    model= model,
    args=training_args,
    train_dataset=dataset,
    formatting_func=formatting_prompts_func,
)
trainer.train()

### 5.保存 LoRA 模型

In [ ]:
# 保存路径
output_dir = "./output/sft/lora"

# 只保存 LoRA 适配器（约几十MB）
model.save_pretrained(output_dir)  # 生成 adapter_config.json + adapter_model.safetensors

## LoRA 模型合并

In [ ]:
# merged_model = model.merge_and_unload()
# merged_model.save_pretrained("./output/sft/lora_merged_model")